- #### Transformaciones
    - ##### sort/orderBy
    - ##### dropDuplicates
    - ##### distinct
    - ##### groupBy
    - ##### union/unionAll
    - ##### unionByName

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_1") \
        .master("local[*]") \
        .getOrCreate()

In [3]:

base_path = "../resources/data/parquet/"
contracts_df = spark.read.parquet(base_path + "contracts_tmp")

contracts_df.show()
contracts_df.printSchema()

+-----------+------------+-----------+----+------------+-------+------------+--------+--------------+------+
|cod_titular|fec_alta_ini|actual_date|diff|fec_alta_fin|cod_iuc|cod_producto| calidad|prioridad_alta|activo|
+-----------+------------+-----------+----+------------+-------+------------+--------+--------------+------+
|      00006|  2012-05-01| 2023-08-15|   8|  2012-05-09|  30000|         100|    baja|          null|  true|
|      00006|  2014-05-01| 2023-08-15|   5|  2014-05-06|  30001|         200|    baja|          null|  true|
|      00006|  2006-02-01| 2023-08-15|   1|  2006-02-02|  30002|         300|    baja|          null|  null|
|      00006|  2012-05-01| 2023-08-15|   3|  2012-05-04|  30003|         150|    baja|          null|  true|
|      00005|  2012-05-01| 2023-08-15|   7|  2012-05-08|  30002|         300|    baja|          null|  true|
|      00006|  2012-05-01| 2023-08-15|   3|  2012-05-04|  30004|         400|   media|          null|  null|
|      00006|  2012

In [10]:
# sort/orderBy

import pyspark.sql.functions as f

contracts_df.orderBy(f.col("cod_iuc").desc(), f.col("fec_alta_fin").desc()).show()

+-----------+------------+-----------+----+------------+-------+------------+--------+--------------+------+
|cod_titular|fec_alta_ini|actual_date|diff|fec_alta_fin|cod_iuc|cod_producto| calidad|prioridad_alta|activo|
+-----------+------------+-----------+----+------------+-------+------------+--------+--------------+------+
|      00003|  2018-10-01| 2023-08-15|   3|  2018-10-04|  30011|        1100|muy alta|            ok|  true|
|      00007|  2014-08-01| 2023-08-15|   7|  2014-08-08|  30010|        1000|    alta|          null|  null|
|      00006|  2014-02-01| 2023-08-15|   5|  2014-02-06|  30010|        1000|    alta|            ok|  true|
|      00006|  2015-09-01| 2023-08-15|   6|  2015-09-07|  30009|         900|    alta|            ok|  true|
|      00002|  2009-10-01| 2023-08-15|   4|  2009-10-05|  30009|         900|    alta|            ok|  true|
|      00002|  2014-03-21| 2023-08-15|   8|  2014-03-29|  30008|         800|    alta|            ok|  true|
|      00006|  2012

In [23]:
# dropDuplicates distinct

#contracts_df.dropDuplicates(["cod_iuc", "cod_producto"]).show()

#contracts_df.distinct().show()

In [37]:
# groupBy -> count, max, min, avg, mean, sum

#contracts_df.groupBy("cod_iuc").count().show()
#contracts_df.groupBy(f.col("cod_iuc")).sum("diff").show()
#contracts_df.groupBy(f.col("cod_iuc")).max("fec_alta_ini").show()

contracts_df.groupBy(f.col("cod_iuc"), f.col("activo"))\
    .agg(
        f.count("*").alias("count"),
        f.sum("diff").alias("sum_diff"),
        f.max("diff").alias("max_diff"),
        f.min("diff").alias("min_diff"),
        f.mean("diff").alias("mean_diff"),
        f.avg("diff").alias("avg_diff"),
        f.max("fec_alta_ini").alias("max_fec_alta_ini")
    ).show()


+-------+------+-----+--------+--------+--------+-----------------+-----------------+----------------+
|cod_iuc|activo|count|sum_diff|max_diff|min_diff|        mean_diff|         avg_diff|max_fec_alta_ini|
+-------+------+-----+--------+--------+--------+-----------------+-----------------+----------------+
|  30002|  null|    2|       7|       6|       1|              3.5|              3.5|      2006-02-01|
|  30006|  null|    1|       6|       6|       6|              6.0|              6.0|      2012-05-01|
|  30001|  true|    3|      16|       9|       2|5.333333333333333|5.333333333333333|      2019-11-10|
|  30003|  true|    5|      23|       7|       2|              4.6|              4.6|      2020-03-01|
|  30002|  true|    1|       7|       7|       7|              7.0|              7.0|      2012-05-01|
|  30000|  true|    2|      13|       8|       5|              6.5|              6.5|      2015-05-25|
|  30007|  null|    1|       0|       0|       0|              0.0|      

In [43]:
contracts_df\
    .groupBy("cod_iuc") \
    .pivot("activo") \
    .agg(
        f.count("*"),
        f.max("fec_alta_fin"),
        f.avg("diff")
    ) \
    .show()

+-------+-------------+----------------------+--------------+-------------+----------------------+-----------------+
|cod_iuc|null_count(1)|null_max(fec_alta_fin)|null_avg(diff)|true_count(1)|true_max(fec_alta_fin)|   true_avg(diff)|
+-------+-------------+----------------------+--------------+-------------+----------------------+-----------------+
|  30002|            2|            2006-02-02|           3.5|            1|            2012-05-08|              7.0|
|  30005|            0|                  null|          null|            2|            2012-05-03|              5.5|
|  30006|            1|            2012-05-07|           6.0|            1|            2016-12-10|              0.0|
|  30009|            0|                  null|          null|            2|            2015-09-07|              5.0|
|  30008|            0|                  null|          null|            3|            2014-03-29|6.333333333333333|
|  30007|            1|            2014-02-01|           0.0|   

In [55]:
## union/unionAll unionByName
contracts_tmp_df = contracts_df.filter(f.col("activo").isNull())

contracts_tmp_df.show()

contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")\
    .union(contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")).show()

contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")\
    .unionByName(contracts_tmp_df.select("actual_date","cod_titular","fec_alta_ini")).show()

+-----------+------------+-----------+----+------------+-------+------------+-------+--------------+------+
|cod_titular|fec_alta_ini|actual_date|diff|fec_alta_fin|cod_iuc|cod_producto|calidad|prioridad_alta|activo|
+-----------+------------+-----------+----+------------+-------+------------+-------+--------------+------+
|      00006|  2006-02-01| 2023-08-15|   1|  2006-02-02|  30002|         300|   baja|          null|  null|
|      00006|  2012-05-01| 2023-08-15|   3|  2012-05-04|  30004|         400|  media|          null|  null|
|      00006|  2012-05-01| 2023-08-15|   6|  2012-05-07|  30006|         600|  media|          null|  null|
|      00006|  2014-02-01| 2023-08-15|   0|  2014-02-01|  30007|         700|   alta|          null|  null|
|      00007|  2001-04-11| 2023-08-15|   6|  2001-04-17|  30002|         300|   baja|          null|  null|
|      00002|  2008-02-01| 2023-08-15|   9|  2008-02-10|  30004|         400|  media|          null|  null|
|      00007|  2014-08-01| 2